<a href="https://colab.research.google.com/github/Lilchoto3/DS-Unit-2-Applied-Modeling/blob/master/module2-wrangle-ml-datasets/LS_DS_232_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Wrangle ML datasets

- [ ] Continue to clean and explore your data. 
- [ ] For the evaluation metric you chose, what score would you get just by guessing?
- [ ] Can you make a fast, first model that beats guessing?

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

**But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset for today's assignment.** Follow the instructions below, to just keep a subset for the Tribeca neighborhood, and remove outliers or dirty data. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!

- Data Source: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)
- Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

In [0]:
# I'm going to keep using my own data for these assignments

# First, I wanna bring in my data and all the changes I made in the last assignment

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')

train = pd.read_csv('https://raw.githubusercontent.com/Lilchoto3/lilchoto3.github.io/master/Destiny2.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Lilchoto3/lilchoto3.github.io/master/Destiny2_Worthy.csv')

train = train.drop(columns='Unnamed: 0')
test = test.drop(columns='Unnamed: 0')

# swap two sets of perks for Duke Mk. 44, since that weapon in particular is weird.
temp = train.ix[229, 'Perk 2 Rolls']
train.ix[229, 'Perk 2 Rolls'] = train.ix[229, 'Perk 3 Rolls']
train.ix[229, 'Perk 3 Rolls'] = temp
# pd.ix is deprecated, but it works over iloc, so I'm fine with it

# remove some unneeded columns
bad_data = [x for x in train.columns if 'Hash' in x or 'Intrinsic' in x or 'Perk 1' in x or 'Perk 2' in x]
train = train.drop(columns=bad_data)
test = test.drop(columns=bad_data)

# de-string-ify some lists to fiddle with their data
def string_to_list(string_in):
  if isinstance(string_in, str):
    list_out = string_in.replace('[','').replace('"',"'").replace(']','').replace(',','').replace("' '","','").split(',')
    for i, string in enumerate(list_out):
      list_out[i] = string.strip("'")
    return list_out
  else:
    return np.NaN
    
train['Perk 3 Rolls'] = train['Perk 3 Rolls'].apply(string_to_list)
train['Perk 4 Rolls'] = train['Perk 4 Rolls'].apply(string_to_list)
test['Perk 3 Rolls'] = test['Perk 3 Rolls'].apply(string_to_list)
test['Perk 4 Rolls'] = test['Perk 4 Rolls'].apply(string_to_list)

# Get a list of all perks 
perklist = []
for i, perk in enumerate(train[['Perk 3 Init','Perk 3 Rolls','Perk 4 Init','Perk 4 Rolls']].values):
  # Check initial perks first
  if isinstance(perk[0], str):
    if perk[0] not in perklist:
      perklist.append(perk[0])
  if isinstance(perk[2], str):
    if perk[2] not in perklist:
      perklist.append(perk[2])
  # Now check the lists
  if isinstance(perk[1], list):
    for p in perk[1]:
      if p not in perklist:
        perklist.append(p)
  if isinstance(perk[3], list):
    for p in perk[3]:
      if p not in perklist:
        perklist.append(p)
for i, perk in enumerate(test[['Perk 3 Init','Perk 3 Rolls','Perk 4 Init','Perk 4 Rolls']].values):
  # Check initial perks first
  if isinstance(perk[0], str):
    if perk[0] not in perklist:
      perklist.append(perk[0])
  if isinstance(perk[2], str):
    if perk[2] not in perklist:
      perklist.append(perk[2])
  # Now check the lists
  if isinstance(perk[1], list):
    for p in perk[1]:
      if p not in perklist:
        perklist.append(p)
  if isinstance(perk[3], list):
    for p in perk[3]:
      if p not in perklist:
        perklist.append(p)

perklist = pd.Series(perklist)

# Drop some bad perks from year 1 guns or exotics that are only for stats
perklist = perklist.drop([69,70,72,74,83,85,86,92,94,96,101,103,106,108,112,120,139])
perklist = perklist.to_list()

# Encode perks as one-hot columns for each weapon
for perk in perklist:
  train['Rolls '+perk] = 0
  test['Rolls '+perk] = 0

def encode_perks(df):
  for i, weapon in enumerate(df.values):
    # How do I see which rolls the weapons have?
    #print(i,train['Perk 3 Rolls'][i])
    # That works, lets get the initial perks first
    if isinstance(df['Perk 3 Init'][i], str):
      if df['Perk 3 Init'][i] in perklist: # Check for bad perks
        df['Rolls '+df['Perk 3 Init'][i]][i] = 1
        #print(train['Rolls '+train['Perk 3 Init'][i]][i])
    if isinstance(df['Perk 4 Init'][i], str):
      if df['Perk 4 Init'][i] in perklist:
        df['Rolls '+df['Perk 4 Init'][i]][i] = 1
        #print(train['Rolls '+train['Perk 4 Init'][i]][i])
    # Now for the lists
    if isinstance(df['Perk 3 Rolls'][i], list):
      for perk in df['Perk 3 Rolls'][i]:
        if perk in perklist:
          df['Rolls '+perk][i] = 1
          #print(train['Rolls '+perk][i])
    if isinstance(df['Perk 4 Rolls'][i], list):
      for perk in df['Perk 4 Rolls'][i]:
        if perk in perklist:
          df['Rolls '+perk][i] = 1
          #print(train['Rolls '+perk][i])

encode_perks(train)
encode_perks(test)

# drop the columns I just encoded
train = train.drop(columns=['Perk 3 Init','Perk 3 Rolls','Perk 4 Init','Perk 4 Rolls'])
test = test.drop(columns=['Perk 3 Init','Perk 3 Rolls','Perk 4 Init','Perk 4 Rolls'])

In [35]:
pd.options.display.max_columns = 175
train.head()

,Name,Weapon Type,Archetype,Rarity,Optimal TTK,Bodyshot TTK,Usage,Impact,Range,Stability,Handling,Reload Speed,Rounds Per Minute,Magazine,Aim Assistance,Zoom,Recoil Direction,Blast Radius,Velocity,Charge Time,Accuracy,Draw Time,Swing Speed,Efficiency,Defense,Ammo Capacity,Rolls Triple Tap,Rolls Hip-Fire Grip,Rolls Grave Robber,Rolls Auto-Loading Holster,Rolls Zen Moment,Rolls Moving Target,Rolls High-Impact Reserves,Rolls Explosive Payload,Rolls Rangefinder,Rolls Genesis,Rolls Opening Shot,Rolls Outlaw,Rolls Rampage,Rolls Dragonfly,Rolls Rapid Hit,Rolls Slideshot,Rolls Eye of the Storm,Rolls Kill Clip,Rolls Swashbuckler,Rolls Threat Detector,Rolls Field Prep,Rolls Full Auto Trigger System,Rolls Pulse Monitor,Rolls Quickdraw,Rolls Snapshot Sights,Rolls Under Pressure,Rolls Dynamic Sway Reduction,Rolls Slideways,Rolls Feeding Frenzy,Rolls Firmly Planted,Rolls Disruption Break,Rolls Shield Disorient,Rolls Underdog,Rolls Ambitious Assassin,Rolls Tap the Trigger,Rolls Mulligan,Rolls Tracking Module,Rolls Demolitionist,Rolls Magnificent Howl,Rolls Timed Payload,Rolls Surrounded,Rolls Headseeker,Rolls Backup Plan,Rolls Onslaught,Rolls Subsistence,Rolls Archer's Tempo,Rolls Explosive Head,Rolls Air Assault,Rolls Sneak Bow,Rolls Multikill Clip,Rolls Shattering Blade,Rolls Whirlwind Blade,Rolls Relentless Strikes,Rolls Tireless Blade,Rolls Counterattack,Rolls En Garde,Rolls Assassin's Blade,Rolls Energy Transfer,Rolls Explosive Light,Rolls Reservoir Burst,Rolls Fourth Time's the Charm,Rolls Reversal of Fortune,Rolls Full Court,Rolls One-Two Punch,Rolls Osmosis,Rolls Overflow,Rolls Firing Line,Rolls Clown Cartridge,Rolls Box Breathing,Rolls Cluster Bomb,Rolls Desperado,Rolls Elemental Capacitor,Rolls Vorpal Weapon,Rolls Archer's Gambit,Rolls Killing Tally,Rolls No Distractions,Rolls Trench Barrel,Rolls Lead from Gold,Rolls Seraph Rounds,Rolls Together Forever,Rolls MIDA Synergy,Rolls Master of Arms,Rolls Meganeura,Rolls Micro-Missile,Rolls Flame Refraction,Rolls Taken Predator,Rolls Firefly,Rolls Parasitism,Rolls Broadhead,Rolls Business Time,Rolls MIDA Radar,Rolls Grenades and Horseshoes,Rolls Longest Winter,Rolls Mechanized Autoloader,Rolls Spread Shot Package,Rolls Supercharged Battery,Rolls Unforeseen Repercussions,Rolls Dark Descent,Rolls Vermin,Rolls Breakthrough,Rolls Leviathan's Sigh,Rolls Play with Your Prey,Rolls Spinning Up,Rolls Sun Blast,Rolls Storm and Stress,Rolls Superconductor,Rolls Bottomless Appetite,Rolls Ionic Return,Rolls Target Acquired,Rolls Lightning Rounds,Rolls Cruel Remedy,Rolls Release the Wolves,Rolls The Roadborn,Rolls Blessing of the Sky,Rolls Death at First Glance,Rolls Thin the Herd,Rolls The Scientific Method,Rolls Long March,Rolls Cosmology,Rolls Last Stand,Rolls Pyrogenesis,Rolls Serve the Colony,Rolls Soul Devourer,Rolls Dynamic Charge,Rolls Penance,Rolls Markov Chain,Rolls The Fundamentals,Rolls Impetus,Rolls For the Empire,Rolls Lightning Rod,Rolls Celerity,Rolls Flash Counter
0,Tango-45 XK5094,Scout Rifle,Precision Frame,Legendary,1.00,1.67,NaN,62.0,50.0,50.0,45.0,44.0,180.0,16.0,65.0,20.0,69.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ancient Gospel,Hand Cannon,Adaptive Frame,Legendary,0.87,1.73,NaN,84.0,47.0,62.0,51.0,49.0,140.0,10.0,74.0,14.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Good Bone Structure,Shotgun,Precision Frame,Legendary,0.00,0.93,NaN,70.0,60.0,50.0,69.0,46.0,65.0,6.0,35.0,12.0,52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,1,1,0,0,1,0,0,0,1,1,1,1

In [36]:
# TODAY'S CODE STARTS HERE

# Admittedly, I kind of jumped the shark yesterday, not really doing so much
# in the way of exploratory data analysis other than using functions
# like pd.head() and those two seaborn graphs at the beginning. Honestly
# I did more of what I would be doing today, yesterday, and most of that
# stems from the fact that I know the data I'm working with, because I _made_
# the data I'm working with.
#
# I was very excited to start doing predictive modelling such that I accidentally
# got ahead of myself, so today I'm going to dial it back a little bit.
#
# Which is to say I'm going to set up the gradient-descent-like RandomSearchCV
# loop I wrote for the Kaggle challenge, and run a ridge regression through it
# for starters.

# First thing's first, though, if I were to guess a baseline value, what would it be?
bc_train = train[['Usage']]
# impute missing values
bc_train = bc_train.fillna(value=0.05)
# log-transform these usages
bc_train['Usage__Log'] = bc_train['Usage'].apply(np.log)
# Get baselines
print('Baseline Mean:',bc_train['Usage'].mean())
print('Baseline Log Mean:',bc_train['Usage__Log'].mean())

Baseline Mean: 0.21143253968253803
Baseline Log Mean: -2.534742161614017


In [37]:
# Okay, that's all fine and dandy, but how accurate is that _really?_
# I'm going to use MAE and R^2 score as my metrics for this
from sklearn.metrics import mean_absolute_error, r2_score
print('Baseline MAE:',mean_absolute_error(bc_train['Usage'], [bc_train['Usage'].mean() for x in bc_train['Usage']]))
print('Log Baseline MAE:',mean_absolute_error(bc_train['Usage__Log'], [bc_train['Usage__Log'].mean() for x in bc_train['Usage__Log']]))
print('Baseline R^2 score:',r2_score(bc_train['Usage'], [bc_train['Usage'].mean() for x in bc_train['Usage']]))
print('Log Baseline R^2 score:',r2_score(bc_train['Usage__Log'], [bc_train['Usage__Log'].mean() for x in bc_train['Usage__Log']]))

Baseline MAE: 0.2595734284454512
Log Baseline MAE: 0.7390476397768057
Baseline R^2 score: 0.0
Log Baseline R^2 score: 0.0


In [38]:
# Right, time to set up the random search and ridge regression
!pip install category_encoders==2.*
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
import category_encoders as ce

In [39]:
# Alright, cool, so now each data frame only has stats that matter to those types of weapons
# the only problem left in terms of NaNs is those in the target
# Since the target is supposed to be the percentage of that weapon's usage in crucible
# and the only weapons with non-nan targets are in the top 100 most used weapons,
# one way I could impute the missing values is take the remaining percentage of usage
# divided by the total number of non-top 100 weapons.
fillna = 100 - train['Usage'].sum()
fillna = fillna / train[train['Usage'].isnull()].shape[0]
fillna

train['Usage'] = train['Usage'].fillna(value=fillna)

# Missing usage values for test data should be zeroes, since the weapons haven't
# been made available to the players yet.
test['Usage'] = test['Usage'].fillna(0)

train.head()

,Name,Weapon Type,Archetype,Rarity,Optimal TTK,Bodyshot TTK,Usage,Impact,Range,Stability,Handling,Reload Speed,Rounds Per Minute,Magazine,Aim Assistance,Zoom,Recoil Direction,Blast Radius,Velocity,Charge Time,Accuracy,Draw Time,Swing Speed,Efficiency,Defense,Ammo Capacity,Rolls Triple Tap,Rolls Hip-Fire Grip,Rolls Grave Robber,Rolls Auto-Loading Holster,Rolls Zen Moment,Rolls Moving Target,Rolls High-Impact Reserves,Rolls Explosive Payload,Rolls Rangefinder,Rolls Genesis,Rolls Opening Shot,Rolls Outlaw,Rolls Rampage,Rolls Dragonfly,Rolls Rapid Hit,Rolls Slideshot,Rolls Eye of the Storm,Rolls Kill Clip,Rolls Swashbuckler,Rolls Threat Detector,Rolls Field Prep,Rolls Full Auto Trigger System,Rolls Pulse Monitor,Rolls Quickdraw,Rolls Snapshot Sights,Rolls Under Pressure,Rolls Dynamic Sway Reduction,Rolls Slideways,Rolls Feeding Frenzy,Rolls Firmly Planted,Rolls Disruption Break,Rolls Shield Disorient,Rolls Underdog,Rolls Ambitious Assassin,Rolls Tap the Trigger,Rolls Mulligan,Rolls Tracking Module,Rolls Demolitionist,Rolls Magnificent Howl,Rolls Timed Payload,Rolls Surrounded,Rolls Headseeker,Rolls Backup Plan,Rolls Onslaught,Rolls Subsistence,Rolls Archer's Tempo,Rolls Explosive Head,Rolls Air Assault,Rolls Sneak Bow,Rolls Multikill Clip,Rolls Shattering Blade,Rolls Whirlwind Blade,Rolls Relentless Strikes,Rolls Tireless Blade,Rolls Counterattack,Rolls En Garde,Rolls Assassin's Blade,Rolls Energy Transfer,Rolls Explosive Light,Rolls Reservoir Burst,Rolls Fourth Time's the Charm,Rolls Reversal of Fortune,Rolls Full Court,Rolls One-Two Punch,Rolls Osmosis,Rolls Overflow,Rolls Firing Line,Rolls Clown Cartridge,Rolls Box Breathing,Rolls Cluster Bomb,Rolls Desperado,Rolls Elemental Capacitor,Rolls Vorpal Weapon,Rolls Archer's Gambit,Rolls Killing Tally,Rolls No Distractions,Rolls Trench Barrel,Rolls Lead from Gold,Rolls Seraph Rounds,Rolls Together Forever,Rolls MIDA Synergy,Rolls Master of Arms,Rolls Meganeura,Rolls Micro-Missile,Rolls Flame Refraction,Rolls Taken Predator,Rolls Firefly,Rolls Parasitism,Rolls Broadhead,Rolls Business Time,Rolls MIDA Radar,Rolls Grenades and Horseshoes,Rolls Longest Winter,Rolls Mechanized Autoloader,Rolls Spread Shot Package,Rolls Supercharged Battery,Rolls Unforeseen Repercussions,Rolls Dark Descent,Rolls Vermin,Rolls Breakthrough,Rolls Leviathan's Sigh,Rolls Play with Your Prey,Rolls Spinning Up,Rolls Sun Blast,Rolls Storm and Stress,Rolls Superconductor,Rolls Bottomless Appetite,Rolls Ionic Return,Rolls Target Acquired,Rolls Lightning Rounds,Rolls Cruel Remedy,Rolls Release the Wolves,Rolls The Roadborn,Rolls Blessing of the Sky,Rolls Death at First Glance,Rolls Thin the Herd,Rolls The Scientific Method,Rolls Long March,Rolls Cosmology,Rolls Last Stand,Rolls Pyrogenesis,Rolls Serve the Colony,Rolls Soul Devourer,Rolls Dynamic Charge,Rolls Penance,Rolls Markov Chain,Rolls The Fundamentals,Rolls Impetus,Rolls For the Empire,Rolls Lightning Rod,Rolls Celerity,Rolls Flash Counter
0,Tango-45 XK5094,Scout Rifle,Precision Frame,Legendary,1.00,1.67,0.033757,62.0,50.0,50.0,45.0,44.0,180.0,16.0,65.0,20.0,69.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ancient Gospel,Hand Cannon,Adaptive Frame,Legendary,0.87,1.73,0.033757,84.0,47.0,62.0,51.0,49.0,140.0,10.0,74.0,14.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Good Bone Structure,Shotgun,Precision Frame,Legendary,0.00,0.93,0.033757,70.0,60.0,50.0,69.0,46.0,65.0,6.0,35.0,12.0,52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,1,1,0,0,

In [40]:
train.isnull().sum()

Name                    0
Weapon Type             0
Archetype               0
Rarity                  0
Optimal TTK             2
                       ..
Rolls Impetus           0
Rolls For the Empire    0
Rolls Lightning Rod     0
Rolls Celerity          0
Rolls Flash Counter     0
Length: 158, dtype: int64

In [41]:
# Before I set up the pipeline, I wanna seperate the training and test data by weapon type
# I need to do this because each weapon type is different in terms of function
# and it doesn't make sense to compare apples to oranges
train_seperated = {}
for wep_type in train['Weapon Type'].unique():
  train_seperated[wep_type] = train[train['Weapon Type']==wep_type]
  train_seperated[wep_type] = train_seperated[wep_type].dropna(how='all', axis=1)

# remove charge time and velocity stats from sidearms because
# Devil's Ruin is a cool gun
train_seperated['Sidearm'] = train_seperated['Sidearm'].drop(columns=['Velocity','Charge Time'])

# remove velocity and blast radius for pulse rifles because of bad stats
# also give claws of the wolf a proper impact stat
train_seperated['Pulse Rifle'].ix[132,'Impact'] = 23
train_seperated['Pulse Rifle'] = train_seperated['Pulse Rifle'].drop(columns=['Velocity','Blast Radius'])

# Give 1 second TTKs for Anarchy and Bastion, because they didn't have any before
train_seperated['Grenade Launcher'].ix[464,'Optimal TTK'] = 1
train_seperated['Grenade Launcher'].ix[464,'Bodyshot TTK'] = 1
train_seperated['Fusion Rifle'].ix[465,'Optimal TTK'] = 1
train_seperated['Fusion Rifle'].ix[465,'Bodyshot TTK'] = 1

test_seperated = {}
for wep_type in test['Weapon Type'].unique():
  test_seperated[wep_type] = test[test['Weapon Type']==wep_type]
  test_seperated[wep_type] = test_seperated[wep_type].dropna(how='all', axis=1)


train_seperated['Auto Rifle'].head()

,Name,Weapon Type,Archetype,Rarity,Optimal TTK,Bodyshot TTK,Usage,Impact,Range,Stability,Handling,Reload Speed,Rounds Per Minute,Magazine,Aim Assistance,Zoom,Recoil Direction,Rolls Triple Tap,Rolls Hip-Fire Grip,Rolls Grave Robber,Rolls Auto-Loading Holster,Rolls Zen Moment,Rolls Moving Target,Rolls High-Impact Reserves,Rolls Explosive Payload,Rolls Rangefinder,Rolls Genesis,Rolls Opening Shot,Rolls Outlaw,Rolls Rampage,Rolls Dragonfly,Rolls Rapid Hit,Rolls Slideshot,Rolls Eye of the Storm,Rolls Kill Clip,Rolls Swashbuckler,Rolls Threat Detector,Rolls Field Prep,Rolls Full Auto Trigger System,Rolls Pulse Monitor,Rolls Quickdraw,Rolls Snapshot Sights,Rolls Under Pressure,Rolls Dynamic Sway Reduction,Rolls Slideways,Rolls Feeding Frenzy,Rolls Firmly Planted,Rolls Disruption Break,Rolls Shield Disorient,Rolls Underdog,Rolls Ambitious Assassin,Rolls Tap the Trigger,Rolls Mulligan,Rolls Tracking Module,Rolls Demolitionist,Rolls Magnificent Howl,Rolls Timed Payload,Rolls Surrounded,Rolls Headseeker,Rolls Backup Plan,Rolls Onslaught,Rolls Subsistence,Rolls Archer's Tempo,Rolls Explosive Head,Rolls Air Assault,Rolls Sneak Bow,Rolls Multikill Clip,Rolls Shattering Blade,Rolls Whirlwind Blade,Rolls Relentless Strikes,Rolls Tireless Blade,Rolls Counterattack,Rolls En Garde,Rolls Assassin's Blade,Rolls Energy Transfer,Rolls Explosive Light,Rolls Reservoir Burst,Rolls Fourth Time's the Charm,Rolls Reversal of Fortune,Rolls Full Court,Rolls One-Two Punch,Rolls Osmosis,Rolls Overflow,Rolls Firing Line,Rolls Clown Cartridge,Rolls Box Breathing,Rolls Cluster Bomb,Rolls Desperado,Rolls Elemental Capacitor,Rolls Vorpal Weapon,Rolls Archer's Gambit,Rolls Killing Tally,Rolls No Distractions,Rolls Trench Barrel,Rolls Lead from Gold,Rolls Seraph Rounds,Rolls Together Forever,Rolls MIDA Synergy,Rolls Master of Arms,Rolls Meganeura,Rolls Micro-Missile,Rolls Flame Refraction,Rolls Taken Predator,Rolls Firefly,Rolls Parasitism,Rolls Broadhead,Rolls Business Time,Rolls MIDA Radar,Rolls Grenades and Horseshoes,Rolls Longest Winter,Rolls Mechanized Autoloader,Rolls Spread Shot Package,Rolls Supercharged Battery,Rolls Unforeseen Repercussions,Rolls Dark Descent,Rolls Vermin,Rolls Breakthrough,Rolls Leviathan's Sigh,Rolls Play with Your Prey,Rolls Spinning Up,Rolls Sun Blast,Rolls Storm and Stress,Rolls Superconductor,Rolls Bottomless Appetite,Rolls Ionic Return,Rolls Target Acquired,Rolls Lightning Rounds,Rolls Cruel Remedy,Rolls Release the Wolves,Rolls The Roadborn,Rolls Blessing of the Sky,Rolls Death at First Glance,Rolls Thin the Herd,Rolls The Scientific Method,Rolls Long March,Rolls Cosmology,Rolls Last Stand,Rolls Pyrogenesis,Rolls Serve the Colony,Rolls Soul Devourer,Rolls Dynamic Charge,Rolls Penance,Rolls Markov Chain,Rolls The Fundamentals,Rolls Impetus,Rolls For the Empire,Rolls Lightning Rod,Rolls Celerity,Rolls Flash Counter
5,The Ringing Nail,Auto Rifle,Precision Frame,Legendary,0.93,1.47,0.033757,29.0,65.0,51.0,51.0,52.0,450.0,32.0,37.0,16.0,69.0,0,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21,Breakneck,Auto Rifle,Precision Frame,Legendary,0.93,1.47,0.033757,29.0,68.0,50.0,43.0,51.0,450.0,34.0,44.0,16.0,80.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27,BrayTech Werewolf,Auto Rifle,Precision Frame,Legendary,0.93,1.47,0.033757,29.0,72.0,47.0,45.0,48.0,450.0,34.0,47.0,16.0,74.0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [47]:
# Okay, well now I can start modelling the data, now that it's imputed and ready.

# set up the pipeline
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    Ridge()
)

# set up the variables that will change as it descends
alpha = {}
for wep_type in train_seperated:
  alpha[wep_type] = 5
param_distributions = {}
for wep_type in train_seperated:
  param_distributions[wep_type] = {}

best_models = {}
prev_scores = {}

for wep_type in train_seperated:
  for x in [5, 4, 3]:
    param_distributions[wep_type]['ridge__alpha'] = [(x/1000000000) for x in range(
        int((alpha[wep_type] - (alpha[wep_type] * x)) * 1000000000),
        int((alpha[wep_type] + (alpha[wep_type] * x)) * 1000000000 + 1),
        int((alpha[wep_type] / 10) * 1000000000)
    )]

    # set up the search
    search = RandomizedSearchCV(
        pipeline,
        param_distributions=param_distributions[wep_type],
        n_iter=10,  # 10 times
        cv=4,       # with 4 validation folds (thank you trace rifles)
        scoring='neg_mean_absolute_error',  # use MAE to score validation
        verbose=10,
        return_train_score=True,
        n_jobs=-1
    )

    # set up targets and features
    # you need to do this because every weapon has different kinds of stats
    target = 'Usage'
    bad_features = ['Name','Weapon Type'] + [target]
    features = train_seperated[wep_type].drop(columns=bad_features).columns
    X_train = train_seperated[wep_type][features]
    #print(X_train.head())
    y_train = train_seperated[wep_type][target]
    #print(y_train.head())

    # fit the search
    print(f'Fitting Search for weapon type {wep_type} of base alpha {alpha[wep_type]} with variance of {x}')
    search.fit(X_train, y_train)

    # Save the best scores and models for later
    if wep_type in best_models:
      if search.best_score_ > prev_scores[wep_type]:
        prev_scores[wep_type] = search.best_score_
        best_models[wep_type] = search
    else:
      prev_scores[wep_type] = search.best_score_
      best_models[wep_type] = search
    
    # update the parameters
    alpha[wep_type] = search.best_params_['ridge__alpha']

    # Show me the best so far
    print('Best Hyperparameters:',search.best_params_)
    print('Cross Validation Accuracy:',search.best_score_)

Fitting Search for weapon type Scout Rifle of base alpha 5 with variance of 5
Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0426s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1257s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0663s.) Setting batch_size=2.


Best Hyperparameters: {'ridge__alpha': 27.5}
Cross Validation Accuracy: -0.18833286841472124
Fitting Search for weapon type Scout Rifle of base alpha 27.5 with variance of 4
Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1320s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0482s.) Setting batch_size=2.


Best Hyperparameters: {'ridge__alpha': 129.25}
Cross Validation Accuracy: -0.1764119248604163
Fitting Search for weapon type Scout Rifle of base alpha 129.25 with variance of 3
Fitting 4 folds for each of 10 candidates, totalling 40 fits


...

[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1591s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0425s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.1s


Best Hyperparameters: {'ridge__alpha': -54.15}
Cross Validation Accuracy: 0.0
Fitting Search for weapon type Trace Rifle of base alpha -54.15 with variance of 3
Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Batch computation too fast (0.1202s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.4s


Best Hyperparameters: {'ridge__alpha': 32.489999999}
Cross Validation Accuracy: 0.0


[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.8s finished


In [0]:
# It's done.
# It works.

# I'll go through the results tomorrow.